In [1]:
import cv2
import numpy as np
import random

In [2]:
def showImg(img, title = str(round(random.random()*1000,0))):
    cv2.imshow(title, img)
    cv2.waitKey(0)
    cv2.destroyWindow(title)

In [3]:
def imageResizeByWidth(img, width=856):
    currRatio = img.shape[0]/img.shape[1]
    print('Shape: ', img.shape)
    height = int(width * currRatio)
    dim = (width, height)
    print('Resized shape: ', dim)
    # resize image
    resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    return resized

In [68]:
fileNamePath = './data/tempEqualize2.jpg'
# fileNamePath = './data/temp2.jpg'

originImg = cv2.imread(fileNamePath)
# showImg(originImg, "Origin image")

Getting Quoc Huy

In [69]:
qh_threshold = 10
minX = 70
minY = 50
qhDiameter = 180  # quoc huy co duong kinh la 14mm

qh = originImg[minY - qh_threshold:minY + qhDiameter + qh_threshold, minX -
               qh_threshold:minX + qhDiameter + qh_threshold]
showImg(qh, "Quoc Huy")

Loading Quoc Huy template

In [80]:
templateQh = cv2.imread('quoc_huy1.jpg')
templateQh = imageResizeByWidth(templateQh, 140)
print("Shape: " ,templateQh.shape)
# showImg(templateQh, 'Template QH')
# cv2.imwrite('qhTemplate.jpg', templateQh)

Shape:  (336, 330, 3)
Resized shape:  (140, 142)
Shape:  (142, 140, 3)


True

#### Quoc huy Shape

In [8]:
qhShape = cv2.imread('quochuy2_shape.jpg',0)
# qhShape = imageResizeByWidth(qhShape, 140)
print("Shape: ", qhShape.shape)

# showImg(qhShape, 'edge QH')

Shape:  (381, 659)


In [9]:

kernel = np.ones((1, 1), np.uint8)
qhShape = cv2.dilate(qhShape, kernel, iterations=1)
# showImg(qhShape, 'edge enhance QH')

qhShape = cv2.GaussianBlur(qhShape, (3, 3), cv2.BORDER_DEFAULT)
# showImg(qhShape, 'edge blur QH')


# Get edges
qhEdges = cv2.Canny(qhShape, 50, 120)
# showImg(qhEdges, 'edge QH')

In [10]:
img = np.zeros(shape=[512, 1000, 3], dtype=np.uint8)

In [11]:
contours, hierarchy = cv2.findContours(qhEdges.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
print ("Number of contours found = ", len(contours))

contours = sorted(contours, key=cv2.contourArea, reverse=True)

# img = qhShape.copy()
cv2.drawContours(img, contours, -1, (255,255,255), 2)

# for c in sorted_contours:
#     cv2.drawContours(orginal_image, [c], -1, (255,155,0), 3)
cv2.waitKey(0)
cv2.imshow('Contours by area', img)

cv2.waitKey(0)
cv2.destroyAllWindows()

Number of contours found =  1


In [12]:
print(cv2.boundingRect(contours[0]))

temp = img[22:324+30,162:316+168]
temp = imageResizeByWidth(temp, 144)
temp = cv2.bitwise_not(temp)
showImg(temp)


(165, 26, 316, 324)
Shape:  (332, 322, 3)
Resized shape:  (144, 148)


In [15]:
cv2.imwrite('qhShape.jpg', temp)

True

#### get edge of origin qh

In [74]:
def getEdgeImage(img):
#     kernel = np.ones((1, 1), np.uint8)
#     img = cv2.dilate(img, kernel, iterations=1)
    # showImg(img, 'edge enhance QH')

    img = cv2.GaussianBlur(img, (5, 5), cv2.BORDER_DEFAULT)
    # showImg(img, 'edge blur QH')


    # Get edges
    edges = cv2.Canny(img, 50, 120)
    # showImg(qhEdges, 'edge QH')
    return edges

In [67]:
qhShape = cv2.imread('qhShape.jpg',0)
# showImg(qhShape, "shape")
ret, thresh1 = cv2.threshold(qhShape, 127, 255, 0)
contours, hierarchy = cv2.findContours(thresh1, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)
print(len(contours))
contours = sorted(contours, key=cv2.contourArea, reverse=True)

templateContour = contours[2]
# print(cv2.boundingRect(contours[0]))
# print(cv2.boundingRect(contours[1]))
# print(cv2.boundingRect(contours[2]))

# cv2.drawContours(qh, [templateContour], -1, (0,255,234), 2)
# showImg(qh,'res1')


3
(0, 0, 144, 148)
(0, 0, 144, 148)
(2, 3, 139, 142)


In [75]:
qh_gray = cv2.cvtColor(qh, cv2.COLOR_BGR2GRAY)
originQhThresh = getEdgeImage(qh_gray)

# ret, originQhThresh = cv2.threshold(qh_gray, 127, 255, 0)
showImg(originQhThresh, "originQhEdges")

# Extract contours from second target image
contours, hierarchy = cv2.findContours(originQhThresh, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)

qhCopy = qh.copy()
cv2.drawContours(qhCopy, contours, -1, (0,255,0), 3)
showImg(qhCopy,'res')


In [76]:

closest_contour = []
for c in contours:
    # Iterate through each contour in the target image and 
    # use cv2.matchShapes to compare contour shapes
    match = cv2.matchShapes(templateContour, c, 3, 0.0)
#     print(match)
    # If the match value is less than 0.15 we
    if match < 0.2:
        closest_contour = c

# print(cv2.boundingRect(closest_contour))

qhCopy = qh.copy()
cv2.drawContours(qhCopy, [closest_contour], -1, (0,255,0), 3)
cv2.imshow('Output', qhCopy)
cv2.waitKey()
cv2.destroyAllWindows()